In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv("insurance_pre.csv")

In [3]:
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [4]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [5]:
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [6]:
dataset.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [7]:
independent=dataset[['age', 'bmi', 'children','sex_male', 'smoker_yes']]

In [8]:
independent

,age,bmi,children,sex_male,smoker_yes
0,19,27.900,0,0,1
1,18,33.770,1,1,0
2,28,33.000,3,1,0
3,33,22.705,0,1,0
4,32,28.880,0,1,0
...,...,...,...,...,...
1333,50,30.970,3,1,0
1334,18,31.920,0,0,0
1335,18,36.850,0,0,0
1336,21,25.800,0,0,0


In [9]:
dependent=dataset[["charges"]]

In [10]:
!pip install lightgbm

In [17]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [3, 10, 20],
    'learning_rate': [0.01, 0.1, 0.3]}
grid = GridSearchCV(LGBMRegressor(), param_grid, refit = True, verbose= 3,n_jobs=-1)
grid.fit(independent, dependent)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000066 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 313
[LightGBM] [Info] Number of data points in the train set: 1338, number of used features: 5
[LightGBM] [Info] Start training from score 13270.422260
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightG

GridSearchCV(estimator=LGBMRegressor(), n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1, 0.3],
                         'max_depth': [3, 10, 20], 'n_estimators': [50, 100]},
             verbose=3)

In [18]:
re=grid.cv_results_
grid_predictions = grid.predict(independent)
from sklearn.metrics import r2_score
r_score=r2_score(dependent,grid_predictions)
print("The R_score value for best parameter {}:".format(grid.best_params_),r_score)


The R_score value for best parameter {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50}: 0.8790230686256992


In [19]:
table=pd.DataFrame.from_dict(re)

In [20]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.483361,0.079844,0.010189,0.004149,0.01,3,50,"{'learning_rate': 0.01, 'max_depth': 3, 'n_est...",0.557956,0.522660,0.533052,0.545102,0.540986,0.539951,0.011818,18
1,0.526178,0.112134,0.008721,0.001316,0.01,3,100,"{'learning_rate': 0.01, 'max_depth': 3, 'n_est...",0.765052,0.712229,0.740688,0.739026,0.742400,0.739879,0.016769,15
2,0.196087,0.025646,0.010810,0.004279,0.01,10,50,"{'learning_rate': 0.01, 'max_depth': 10, 'n_es...",0.565258,0.530237,0.540971,0.548945,0.539703,0.545023,0.011732,16
3,0.301301,0.016855,0.010748,0.003444,0.01,10,100,"{'learning_rate': 0.01, 'max_depth': 10, 'n_es...",0.767177,0.715397,0.751758,0.744336,0.739562,0.743646,0.016939,14
4,0.171990,0.013266,0.008683,0.003433,0.01,20,50,"{'learning_rate': 0.01, 'max_depth': 20, 'n_es...",0.565258,0.530237,0.540971,0.548945,0.539703,0.545023,0.011732,16
5,0.312816,0.021849,0.011407,0.001807,0.01,20,100,"{'learning_rate': 0.01, 'max_depth': 20, 'n_es...",0.767177,0.715397,0.751766,0.744336,0.739562,0.743648,0.016940,13
6,0.236406,0.073101,0.012504,0.003009,0.10,3,50,"{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",0.885009,0.815430,0.894452,0.852458,0.866767,0.862823,0.027791,1
7,0.525105,0.094452,0.013570,0.002764,0.10,3,100,"{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",0.881497,0.811597,0.893434,0.851057,0.860707,0.859658,0.028294,2
8,0.223932,0.013761,0.009920,0.002527,0.10,10,50,"{'learning_rate': 0.1, 'max_depth': 10, 'n_est...",0.869225,0.796852,0.888157,0.839929,0.858234,0.850479,0.031043,4
9,0.452772,0.025466,0.011434,0.000889,0.10,10,100,"{'learning_rate': 0.1, 'max_depth': 10, 'n_est...",0.862049,0.787063,0.878208,0.836449,0.848552,0.842464,0.030996,7


In [21]:
age_input=float(input("Age:"))
bmi_input=float(input("BMI:"))
children_input=float(input("Children:"))
sex_male_input=int(input("Sex Male 0 or 1:"))
smoker_yes_input=int(input("Smoker Yes 0 or 1:"))

Age: 25
BMI: 25
Children: 0
Sex Male 0 or 1: 0
Smoker Yes 0 or 1: 0


In [22]:
Insurance_charges=grid.predict([[age_input,bmi_input,children_input,sex_male_input,smoker_yes_input]])
print("Insurance Charges={}".format(Insurance_charges))


Insurance Charges=[4487.72273439]
